In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from IPython.display import display, Markdown, Math, HTML
import matplotlib.pyplot as plt
from scipy.stats import t

# Define the general polynomial model of degree n
# def polynomial_model(x, *coeffs):
#     return sum(c * x**i for i, c in enumerate(coeffs))

def polynomial_model(x, *params):
    return sum([p * (x ** i) for i, p in enumerate(params)])

# Function to calculate and return goodness-of-fit parameters
def fit_poly(x, y, degree):
    # Perform curve fitting
    initial_guess = np.ones(degree + 1)  # Initial guess for the coefficients
    popt, _ = curve_fit(polynomial_model, x, y, p0=initial_guess)
    
    # Calculate the residuals
    residuals = y - polynomial_model(x, *popt)

    # Calculate the total sum of squares (TSS)
    ss_tot = np.sum((y - np.mean(y)) ** 2)

    # Calculate the residual sum of squares (RSS)
    ss_res = np.sum(residuals ** 2)

    # Calculate R-squared
    r_squared = 1 - (ss_res / ss_tot)

    # Calculate the reduced chi-squared
    # Degrees of freedom: number of observations - number of parameters
    degrees_of_freedom = len(x) - len(popt)
    reduced_chi_squared = ss_res / degrees_of_freedom

    # Data to be written
    fit_data = {
        'Optimized parameters': popt,
        "R-squared": r_squared,
        "Reduced chi-squared": reduced_chi_squared
    }

    # Display the formatted output in the Jupyter Notebook cell
    output = "# Goodness-of-fit parameters\n"
    for key, value in fit_data.items():
        output += f'\n{key}: {value}\n'

    return output, popt

# Function to format coefficients in scientific notation for LaTeX
def format_coefficient(value):
    sci = "{:.5e}".format(value).split('e')
    base = sci[0]
    exponent = int(sci[1])
    sign = "-" if base.startswith('-') else "+"
    base = base.lstrip('-')
    return f"{sign} {base} \\times 10^{{{exponent}}}"

# Generalized function to display fitting parameters and LaTeX equation
def display_fitting(material_name, property_name, output, popt):
    # Print the output
    display(HTML("<hr>"))
    text = f'**Fitting parameters for {material_name}** \n'
    display(Markdown(text))
    print(output)

    # Format the coefficients
    formatted_coeffs = [format_coefficient(c) for c in popt]

    # Handle T^0 term separately
    equation_terms = [formatted_coeffs[0]]  # Start with the constant term

    # Create the polynomial equation in LaTeX format for terms T^i where i > 0
    for i in range(1, len(popt)):
        term = formatted_coeffs[i]
        if term.startswith('+'):
            equation_terms.append(f"{term} T^{i}")
        else:
            equation_terms.append(f"{term} T^{i}")

    # Join the terms without adding an extra plus sign at the beginning
    equation = rf'{property_name} = ' + ' '.join(equation_terms).replace(' + -', ' -')

    # Display the equation using LaTeX formatting with the formatted coefficients
    print(f'The equation for {material_name} {property_name} is:\n')
    # display(Math(equation))
     # Display the equation using LaTeX formatting with the formatted coefficients inside a box
    boxed_equation = f"""
    <div style="border: 2px solid black; padding: 10px; display: inline-block;">
        $$ {equation} $$
    </div>
    """
    display(HTML(boxed_equation))
    print('\n\n')

def fit_poly_with_confidence(x, y, degree, confidence=0.95):
    # Perform curve fitting
    initial_guess = np.ones(degree + 1)  # Initial guess for the coefficients
    try:
        popt, pcov = curve_fit(polynomial_model, x, y, p0=initial_guess)
        
        # Calculate the mean fit
        mean_fit = polynomial_model(x, *popt)
         
        # Calculate the residuals
        residuals = y - mean_fit
    
        # Calculate the total sum of squares (TSS)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
      
        # Calculate the residual sum of squares (RSS)
        ss_res = np.sum(residuals ** 2)
      
        # Calculate R-squared
        r_squared = 1 - (ss_res / ss_tot)
      
        # Calculate the reduced chi-squared
        # Degrees of freedom: number of observations - number of parameters
        degrees_of_freedom = len(x) - len(popt)
        reduced_chi_squared = ss_res / degrees_of_freedom
       
        # Calculate the 95% confidence intervals for the fit parameters
        alpha = 1.0 - confidence
        dof = max(0, degrees_of_freedom)
        t_val = t.ppf(1.0 - alpha / 2., dof)
        ci = t_val * np.sqrt(np.diag(pcov))
      
        # Calculate the prediction intervals
        sum_squared_errors = np.sum(residuals**2)
        mean_x = np.mean(x)
        n = len(x)
        t_val = t.ppf((1.0 + confidence) / 2.0, dof)
        
        pred_interval = []
        for i in range(len(x)):
            s_err = np.sqrt(1/n + (x[i] - mean_x)**2 / np.sum((x - mean_x)**2))
            margin = t_val * np.sqrt(sum_squared_errors / dof) * s_err
            pred_interval.append(margin)

        pred_interval = np.array(pred_interval)
        lower_pred = mean_fit - pred_interval
        upper_pred = mean_fit + pred_interval

        # Data to be written
        fit_data = {
            'Optimized parameters': popt,
            "R-squared": r_squared,
            "Reduced chi-squared": reduced_chi_squared,
            "Confidence intervals": ci,
            "Mean fit": mean_fit,
            "Lower 95% prediction": lower_pred,
            "Upper 95% prediction": upper_pred
        }

    except Exception as e:
        print("Exception occurred:", e)
        fit_data = {
            'Optimized parameters': None,
            "R-squared": None,
            "Reduced chi-squared": None,
            "Confidence intervals": None,
            "Mean fit": None,
            "Lower 95% prediction": None,
            "Upper 95% prediction": None,
            "Error": str(e)
        }

    return fit_data

